# Get Data

In [18]:
import pandas as pd
import ast

# dataLamaPath = 'data/new-fix-data.csv'
dataBaruPath = 'data/tempat-wisata-di-yogyakarta-overview.csv'
dataByPlace = 'data/byPlaceData.csv'

# dfOld = pd.read_csv(dataLamaPath)
dfNew = pd.read_csv(dataBaruPath)

dfByPlace = pd.read_csv(dataByPlace)

In [19]:
dfNew.head()

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,website,can_claim,owner_name,owner_profile_link,featured_image,workday_timing,is_temporarily_closed,closed_on,phone,review_keywords
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,*****
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,*****
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,*****
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,*****
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,*****


In [20]:
dfByPlace.head()

,Unnamed: 0,name,address,coordinates,workday_timing,closed_on,reviews,rating,most_popular_times,popular_times,reviewer_name,rating_review,review_text,published_at_date,accessibility_enabled,planning_enabled,children_enabled,is_rating_updated,is_reviews_updated
0,0,( Alun Alun Utara),"59W8+F66 ( Alun Alun Utara), Jl. Alun-Alun Ut...","-7.8038427,110.3655155",Buka 24 jam,Open All Days,43928,4.8,Not Present,Not Present,"['fandy achmad hamid', 'Lucky Zacky']","[4.0, 5.0]",['Tempat Nongkrong yang enak dikunjungi bareng...,"['2023-07-03 18:57:13.071', '2023-11-07 08:23:...","Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak,1.0,1.0
1,1,Embung Kladuan UII,"Embung Kladuan UII, Moh. Hatta, Lodadi, Umbul...","-7.6889138,110.4159846",Buka 24 jam,Open All Days,5101,4.7,"08.00, 09.00, 11.00\n---\nTime Label: 08.00\nA...",Monday:\n 04.00: 50% | Biasanya tidak terla...,"['Rizky D. Novyantika', 'Rahajeng Anindyajati'...","[4.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0]","['Embung ini sebenernya lumayan buat nyore, ta...","['2022-10-05 10:41:56.075', '2022-01-01 10:55:...","Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak,1.0,1.0
2,2,Jembatan bok bang,"6PH9+4W8 Jembatan bok bang, Unnamed Road, Are...","-7.7722092,110.7198427",Buka 24 jam,Open All Days,17765,4.6,"23.00, 22.00, 00.00\n---\nTime Label: 23.00\nA...",Monday:\n 04.00: 0% | Idle\n 05.00: 0% |...,['Boneka Mainan'],[5.0],"['Saat saya memancing, saya sangat senang. Pa...",['2021-02-19 11:02:46.654'],Tempat parkir khusus pengguna kursi roda,Tidak menerima reservasi,Tidak cocok untuk anak-anak,1.0,1.0
3,3,Sendang Sari Mulyo,"2CMC+GCH Sendang Sari Mulyo, Unnamed Road, No...","-7.9661915,110.4211151",08.00-16.00,Open All Days,12,4.9,Not Present,Not Present,"['Muhammad Zaidan Robbani', 'Handika Roma', 'F...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0]",['Ada sentra pembuatan bawang goreng jugaaa lo...,"['2021-07-03 05:39:15.538', '2022-11-02 02:04:...","Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak,0.0,0.0
4,4,Stupa Buddha Dawangsari,"6FGX+926 Stupa Buddha Dawangsari, Dawang Sari...","-7.7740887999999995,110.4975261",08.00-16.00,Open All Days,16,4.9,Not Present,Not Present,"['Keperluan Penelitian', 'Dini Suryaningsih', ...","[5.0, 4.0, 3.0, 4.0, 4.0, 4.0, 5.0, 5.0]",['Candi Dawangsari merupakan peninggalan Stupa...,"['2023-10-29 00:06:11.689', '2023-05-24 12:39:...",Tempat parkir khusus pengguna kursi roda,Tidak menerima reservasi,Tidak cocok untuk anak-anak,1.0,1.0


# Data Assessing

In [15]:
# Data Baru
dfNew.columns

Index(['place_id', 'name', 'reviews', 'main_category', 'categories', 'rating',
       'address', 'link', 'query', 'description', 'is_spending_on_ads',
       'competitors', 'website', 'can_claim', 'owner_name',
       'owner_profile_link', 'featured_image', 'workday_timing',
       'is_temporarily_closed', 'closed_on', 'phone', 'review_keywords'],
      dtype='object')

In [16]:
dfByPlace.columns

Index(['Unnamed: 0', 'name', 'address', 'coordinates', 'workday_timing',
       'closed_on', 'reviews', 'rating', 'most_popular_times', 'popular_times',
       'reviewer_name', 'rating_review', 'review_text', 'published_at_date',
       'accessibility_enabled', 'planning_enabled', 'children_enabled',
       'is_rating_updated', 'is_reviews_updated'],
      dtype='object')

Data yang diambil: 
- name
- address
- coordinates
- workday_timing
- closed_on
- reviews
- rating
- most_popular_times
- popular_times
- reviewer_name
- rating_review
- review_text
- published_at_date
- accessibility_enabled
- planning_enabled
- children_enabled

## Cek Informasi Kolom

## Merampingkan Data

In [ ]:
# grpByReviewer = dfOld.groupby('name').agg({
#     'address': lambda x: list(x),
#     'coordinates': lambda x: list(x),
#     'workday_timing': lambda x: list(x),
#     'closed_on': lambda x: list(x),
#     'reviews': lambda x: list(x),
#     'rating': lambda x: list(x),
#     'most_popular_times': lambda x: list(x),
#     'popular_times': lambda x: list(x),
#     'popular_times': lambda x: list(x),
#     'reviewer_name': lambda x: list(x),
#     'rating_review': lambda x: list(x),
#     'review_text': lambda x: list(x),
#     'published_at_date': lambda x: list(x),
#     'accessibility_enabled': lambda x: list(x),
#     'planning_enabled': lambda x: list(x),
#     'children_enabled': lambda x: list(x)
# }).reset_index()

# grpByReviewer.to_csv('data/byPlaceData.csv', index=False)

## Normalisasi Bentuk

Percobaan

In [ ]:
# def process_lists_in_dataframe(df, columns_to_normalize):
#     # Loop melalui setiap kolom yang perlu dinormalisasi dalam DataFrame
#     for column in columns_to_normalize:
#         if column in df.columns:
#             # Loop melalui setiap baris dalam kolom yang relevan
#             for i in range(len(df[column])):
#                 if isinstance(df[column].iloc[i], str):
#                     try:
#                         # Convert string representation of list to actual list
#                         df[column].iloc[i] = ast.literal_eval(df[column].iloc[i])
#                     except (ValueError, SyntaxError):
#                         continue
                
#                 if isinstance(df[column].iloc[i], list):
#                     # Convert list to a set to check for duplicates
#                     value_counts = pd.Series(df[column].iloc[i]).value_counts()
                    
#                     # Check if there are any duplicates (i.e., any value count >= 2)
#                     if any(value_counts >= 1):
#                         # Replace the list with the first element
#                         df.at[i, column] = df[column].iloc[i][0]
    
#     return df

# columns_to_normalize = ['address', 'coordinates', 'workday_timing', 'closed_on','rating',
#                         'reviews', 'most_popular_times', 'popular_times', 
#                         'accessibility_enabled', 'planning_enabled', 'children_enabled']

# dfByPlace = process_lists_in_dataframe(dfByPlace, columns_to_normalize)

## Ambil Data 

### Coordinates 

In [24]:
def update_rating(data1, data2):
    # Loop melalui setiap baris di data1
    for idx, row in data1.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data2[data2['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'coordinates'] = matching_row['coordinates'].values[0]
        else:
            print('Tidak ada data', row['name'])
    
    # Kembalikan data1 yang sudah diperbarui
    return data1

updatedDf = update_rating(dfNew, dfByPlace)
updatedDf.head()


Ada data Titik Nol Yogyakarta
Ada data Kawasan Wisata Malioboro
Ada data Museum Benteng Vredeburg
Tidak ada data Kampung Wisata Taman Sari
Ada data Kampung Wisata Sayidan
Ada data Gereja Sayidan
Ada data Plengkung Gading
Ada data Embung Langensari
Ada data Taman Pintar Yogyakarta
Ada data Taman Pelangi Jogja
Tidak ada data KEBON BINATANG
Tidak ada data Keraton Ngayogyakarta Hadiningrat
Tidak ada data Kraton Jogja
Ada data Bringin Kembar Alun-Alun Selatan (Masangin) Yogyakarta
Tidak ada data Alun-Alun Lor Yogyakarta
Ada data DINO Adventure
Ada data Kampung Wisata Rejowinangun
Tidak ada data nan
Ada data Taman Kearifan UGM
Ada data Slasar Malioboro
Ada data Kampung Wisata Sosromenduran
Ada data Kampung Wisata Tahunan
Ada data Monumen Batik Yogyakarta
Ada data Jembatan Sayidan
Ada data Malioboro Yogyakarta Indonesia
Ada data Tebing Breksi
Ada data Bukit Bintang
Tidak ada data Puncak Pinus Becici
Ada data Bendhung Lepen
Ada data Candi Prambanan
Ada data Makam Panembahan Senopati
Ada data w

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,can_claim,owner_name,owner_profile_link,featured_image,workday_timing,is_temporarily_closed,closed_on,phone,review_keywords,coordinates
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,"-7.8013965999999995,110.36476409999999"
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,"-7.798248699999999,110.3652793"
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,"-7.800293,110.3661642"
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,NaN
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,"-7.802505999999999,110.37030759999999"


### Workday Timing

In [25]:
def update_rating(data1, data2):
    # Loop melalui setiap baris di data1
    for idx, row in data1.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data2[data2['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'workday_timing'] = matching_row['workday_timing'].values[0]
        else:
            print('Tidak ada data', row['name'])
    
    # Kembalikan data1 yang sudah diperbarui
    return data1

updatedDf = update_rating(dfNew, dfByPlace)
updatedDf.head()


Ada data Titik Nol Yogyakarta
Ada data Kawasan Wisata Malioboro
Ada data Museum Benteng Vredeburg
Tidak ada data Kampung Wisata Taman Sari
Ada data Kampung Wisata Sayidan
Ada data Gereja Sayidan
Ada data Plengkung Gading
Ada data Embung Langensari
Ada data Taman Pintar Yogyakarta
Ada data Taman Pelangi Jogja
Tidak ada data KEBON BINATANG
Tidak ada data Keraton Ngayogyakarta Hadiningrat
Tidak ada data Kraton Jogja
Ada data Bringin Kembar Alun-Alun Selatan (Masangin) Yogyakarta
Tidak ada data Alun-Alun Lor Yogyakarta
Ada data DINO Adventure
Ada data Kampung Wisata Rejowinangun
Tidak ada data nan
Ada data Taman Kearifan UGM
Ada data Slasar Malioboro
Ada data Kampung Wisata Sosromenduran
Ada data Kampung Wisata Tahunan
Ada data Monumen Batik Yogyakarta
Ada data Jembatan Sayidan
Ada data Malioboro Yogyakarta Indonesia
Ada data Tebing Breksi
Ada data Bukit Bintang
Tidak ada data Puncak Pinus Becici
Ada data Bendhung Lepen
Ada data Candi Prambanan
Ada data Makam Panembahan Senopati
Ada data w

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,can_claim,owner_name,owner_profile_link,featured_image,workday_timing,is_temporarily_closed,closed_on,phone,review_keywords,coordinates
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,Buka 24 jam,*****,*****,*****,*****,"-7.8013965999999995,110.36476409999999"
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,Buka 24 jam,*****,*****,*****,*****,"-7.798248699999999,110.3652793"
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,08.00-15.30,*****,*****,*****,*****,"-7.800293,110.3661642"
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,NaN
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,Buka 24 jam,*****,*****,*****,*****,"-7.802505999999999,110.37030759999999"


### Closed On

In [26]:
def update_rating(data1, data2):
    # Loop melalui setiap baris di data1
    for idx, row in data1.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data2[data2['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'closed_on'] = matching_row['closed_on'].values[0]
        else:
            print('Tidak ada data', row['name'])
    
    # Kembalikan data1 yang sudah diperbarui
    return data1

updatedDf = update_rating(dfNew, dfByPlace)
updatedDf.head()


Ada data Titik Nol Yogyakarta
Ada data Kawasan Wisata Malioboro
Ada data Museum Benteng Vredeburg
Tidak ada data Kampung Wisata Taman Sari
Ada data Kampung Wisata Sayidan
Ada data Gereja Sayidan
Ada data Plengkung Gading
Ada data Embung Langensari
Ada data Taman Pintar Yogyakarta
Ada data Taman Pelangi Jogja
Tidak ada data KEBON BINATANG
Tidak ada data Keraton Ngayogyakarta Hadiningrat
Tidak ada data Kraton Jogja
Ada data Bringin Kembar Alun-Alun Selatan (Masangin) Yogyakarta
Tidak ada data Alun-Alun Lor Yogyakarta
Ada data DINO Adventure
Ada data Kampung Wisata Rejowinangun
Tidak ada data nan
Ada data Taman Kearifan UGM
Ada data Slasar Malioboro
Ada data Kampung Wisata Sosromenduran
Ada data Kampung Wisata Tahunan
Ada data Monumen Batik Yogyakarta
Ada data Jembatan Sayidan
Ada data Malioboro Yogyakarta Indonesia
Ada data Tebing Breksi
Ada data Bukit Bintang
Tidak ada data Puncak Pinus Becici
Ada data Bendhung Lepen
Ada data Candi Prambanan
Ada data Makam Panembahan Senopati
Ada data w

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,can_claim,owner_name,owner_profile_link,featured_image,workday_timing,is_temporarily_closed,closed_on,phone,review_keywords,coordinates
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.8013965999999995,110.36476409999999"
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.798248699999999,110.3652793"
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,08.00-15.30,*****,Senin,*****,*****,"-7.800293,110.3661642"
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,*****,NaN
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.802505999999999,110.37030759999999"


### most_popular_times

In [27]:
def update_rating(data1, data2):
    # Loop melalui setiap baris di data1
    for idx, row in data1.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data2[data2['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'most_popular_times'] = matching_row['most_popular_times'].values[0]
        else:
            print('Tidak ada data', row['name'])
    
    # Kembalikan data1 yang sudah diperbarui
    return data1

updatedDf = update_rating(dfNew, dfByPlace)
updatedDf.head()

Ada data Titik Nol Yogyakarta
Ada data Kawasan Wisata Malioboro
Ada data Museum Benteng Vredeburg
Tidak ada data Kampung Wisata Taman Sari
Ada data Kampung Wisata Sayidan
Ada data Gereja Sayidan
Ada data Plengkung Gading
Ada data Embung Langensari
Ada data Taman Pintar Yogyakarta
Ada data Taman Pelangi Jogja
Tidak ada data KEBON BINATANG
Tidak ada data Keraton Ngayogyakarta Hadiningrat
Tidak ada data Kraton Jogja
Ada data Bringin Kembar Alun-Alun Selatan (Masangin) Yogyakarta
Tidak ada data Alun-Alun Lor Yogyakarta
Ada data DINO Adventure
Ada data Kampung Wisata Rejowinangun
Tidak ada data nan
Ada data Taman Kearifan UGM
Ada data Slasar Malioboro
Ada data Kampung Wisata Sosromenduran
Ada data Kampung Wisata Tahunan
Ada data Monumen Batik Yogyakarta
Ada data Jembatan Sayidan
Ada data Malioboro Yogyakarta Indonesia
Ada data Tebing Breksi
Ada data Bukit Bintang
Tidak ada data Puncak Pinus Becici
Ada data Bendhung Lepen
Ada data Candi Prambanan
Ada data Makam Panembahan Senopati
Ada data w

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,owner_name,owner_profile_link,featured_image,workday_timing,is_temporarily_closed,closed_on,phone,review_keywords,coordinates,most_popular_times
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.8013965999999995,110.36476409999999","20.00, 21.00, 19.00\n---\nTime Label: 20.00\nA..."
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.798248699999999,110.3652793","21.00, 20.00, 19.00\n---\nTime Label: 21.00\nA..."
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,08.00-15.30,*****,Senin,*****,*****,"-7.800293,110.3661642","11.00, 12.00, 13.00\n---\nTime Label: 11.00\nA..."
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,*****,NaN,NaN
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.802505999999999,110.37030759999999","19.00, 20.00, 18.00\n---\nTime Label: 19.00\nA..."


### popular_times

In [28]:
def update_rating(data1, data2):
    # Loop melalui setiap baris di data1
    for idx, row in data1.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data2[data2['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'popular_times'] = matching_row['popular_times'].values[0]
        else:
            print('Tidak ada data', row['name'])
    
    # Kembalikan data1 yang sudah diperbarui
    return data1

updatedDf = update_rating(dfNew, dfByPlace)
updatedDf.head()

Ada data Titik Nol Yogyakarta
Ada data Kawasan Wisata Malioboro
Ada data Museum Benteng Vredeburg
Tidak ada data Kampung Wisata Taman Sari
Ada data Kampung Wisata Sayidan
Ada data Gereja Sayidan
Ada data Plengkung Gading
Ada data Embung Langensari
Ada data Taman Pintar Yogyakarta
Ada data Taman Pelangi Jogja
Tidak ada data KEBON BINATANG
Tidak ada data Keraton Ngayogyakarta Hadiningrat
Tidak ada data Kraton Jogja
Ada data Bringin Kembar Alun-Alun Selatan (Masangin) Yogyakarta
Tidak ada data Alun-Alun Lor Yogyakarta
Ada data DINO Adventure
Ada data Kampung Wisata Rejowinangun
Tidak ada data nan
Ada data Taman Kearifan UGM
Ada data Slasar Malioboro
Ada data Kampung Wisata Sosromenduran
Ada data Kampung Wisata Tahunan
Ada data Monumen Batik Yogyakarta
Ada data Jembatan Sayidan
Ada data Malioboro Yogyakarta Indonesia
Ada data Tebing Breksi
Ada data Bukit Bintang
Tidak ada data Puncak Pinus Becici
Ada data Bendhung Lepen
Ada data Candi Prambanan
Ada data Makam Panembahan Senopati
Ada data w

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,owner_profile_link,featured_image,workday_timing,is_temporarily_closed,closed_on,phone,review_keywords,coordinates,most_popular_times,popular_times
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.8013965999999995,110.36476409999999","20.00, 21.00, 19.00\n---\nTime Label: 20.00\nA...",Monday:\n 04.00: 2% | Biasanya tidak ramai\...
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.798248699999999,110.3652793","21.00, 20.00, 19.00\n---\nTime Label: 21.00\nA...",Monday:\n 04.00: 1% | Biasanya tidak ramai\...
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,*****,*****,08.00-15.30,*****,Senin,*****,*****,"-7.800293,110.3661642","11.00, 12.00, 13.00\n---\nTime Label: 11.00\nA...",Tuesday:\n 06.00: 0% | Idle\n 07.00: 0% ...
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,*****,NaN,NaN,NaN
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.802505999999999,110.37030759999999","19.00, 20.00, 18.00\n---\nTime Label: 19.00\nA...",Monday:\n 04.00: 29% | Biasanya tidak terla...


### accessibility_enabled

In [29]:
def update_rating(data1, data2):
    # Loop melalui setiap baris di data1
    for idx, row in data1.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data2[data2['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'accessibility_enabled'] = matching_row['accessibility_enabled'].values[0]
        else:
            print('Tidak ada data', row['name'])
    
    # Kembalikan data1 yang sudah diperbarui
    return data1

updatedDf = update_rating(dfNew, dfByPlace)
updatedDf.head()

Ada data Titik Nol Yogyakarta
Ada data Kawasan Wisata Malioboro
Ada data Museum Benteng Vredeburg
Tidak ada data Kampung Wisata Taman Sari
Ada data Kampung Wisata Sayidan
Ada data Gereja Sayidan
Ada data Plengkung Gading
Ada data Embung Langensari
Ada data Taman Pintar Yogyakarta
Ada data Taman Pelangi Jogja
Tidak ada data KEBON BINATANG
Tidak ada data Keraton Ngayogyakarta Hadiningrat
Tidak ada data Kraton Jogja
Ada data Bringin Kembar Alun-Alun Selatan (Masangin) Yogyakarta
Tidak ada data Alun-Alun Lor Yogyakarta
Ada data DINO Adventure
Ada data Kampung Wisata Rejowinangun
Tidak ada data nan
Ada data Taman Kearifan UGM
Ada data Slasar Malioboro
Ada data Kampung Wisata Sosromenduran
Ada data Kampung Wisata Tahunan
Ada data Monumen Batik Yogyakarta
Ada data Jembatan Sayidan
Ada data Malioboro Yogyakarta Indonesia
Ada data Tebing Breksi
Ada data Bukit Bintang
Tidak ada data Puncak Pinus Becici
Ada data Bendhung Lepen
Ada data Candi Prambanan
Ada data Makam Panembahan Senopati
Ada data w

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,featured_image,workday_timing,is_temporarily_closed,closed_on,phone,review_keywords,coordinates,most_popular_times,popular_times,accessibility_enabled
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.8013965999999995,110.36476409999999","20.00, 21.00, 19.00\n---\nTime Label: 20.00\nA...",Monday:\n 04.00: 2% | Biasanya tidak ramai\...,"Pintu masuk khusus pengguna kursi roda, Tempat..."
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.798248699999999,110.3652793","21.00, 20.00, 19.00\n---\nTime Label: 21.00\nA...",Monday:\n 04.00: 1% | Biasanya tidak ramai\...,"Pintu masuk khusus pengguna kursi roda, Tempat..."
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,*****,08.00-15.30,*****,Senin,*****,*****,"-7.800293,110.3661642","11.00, 12.00, 13.00\n---\nTime Label: 11.00\nA...",Tuesday:\n 06.00: 0% | Idle\n 07.00: 0% ...,"Kursi khusus pengguna kursi roda, Pintu masuk ..."
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,*****,NaN,NaN,NaN,NaN
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,Buka 24 jam,*****,Open All Days,*****,*****,"-7.802505999999999,110.37030759999999","19.00, 20.00, 18.00\n---\nTime Label: 19.00\nA...",Monday:\n 04.00: 29% | Biasanya tidak terla...,"Pintu masuk khusus pengguna kursi roda, Tempat..."


### planning_enabled

In [30]:
def update_rating(data1, data2):
    # Loop melalui setiap baris di data1
    for idx, row in data1.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data2[data2['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'planning_enabled'] = matching_row['planning_enabled'].values[0]
        else:
            print('Tidak ada data', row['name'])
    
    # Kembalikan data1 yang sudah diperbarui
    return data1

updatedDf = update_rating(dfNew, dfByPlace)
updatedDf.head()

Ada data Titik Nol Yogyakarta
Ada data Kawasan Wisata Malioboro
Ada data Museum Benteng Vredeburg
Tidak ada data Kampung Wisata Taman Sari
Ada data Kampung Wisata Sayidan
Ada data Gereja Sayidan
Ada data Plengkung Gading
Ada data Embung Langensari
Ada data Taman Pintar Yogyakarta
Ada data Taman Pelangi Jogja
Tidak ada data KEBON BINATANG
Tidak ada data Keraton Ngayogyakarta Hadiningrat
Tidak ada data Kraton Jogja
Ada data Bringin Kembar Alun-Alun Selatan (Masangin) Yogyakarta
Tidak ada data Alun-Alun Lor Yogyakarta
Ada data DINO Adventure
Ada data Kampung Wisata Rejowinangun
Tidak ada data nan
Ada data Taman Kearifan UGM
Ada data Slasar Malioboro
Ada data Kampung Wisata Sosromenduran
Ada data Kampung Wisata Tahunan
Ada data Monumen Batik Yogyakarta
Ada data Jembatan Sayidan
Ada data Malioboro Yogyakarta Indonesia
Ada data Tebing Breksi
Ada data Bukit Bintang
Tidak ada data Puncak Pinus Becici
Ada data Bendhung Lepen
Ada data Candi Prambanan
Ada data Makam Panembahan Senopati
Ada data w

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,workday_timing,is_temporarily_closed,closed_on,phone,review_keywords,coordinates,most_popular_times,popular_times,accessibility_enabled,planning_enabled
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,Buka 24 jam,*****,Open All Days,*****,*****,"-7.8013965999999995,110.36476409999999","20.00, 21.00, 19.00\n---\nTime Label: 20.00\nA...",Monday:\n 04.00: 2% | Biasanya tidak ramai\...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,Buka 24 jam,*****,Open All Days,*****,*****,"-7.798248699999999,110.3652793","21.00, 20.00, 19.00\n---\nTime Label: 21.00\nA...",Monday:\n 04.00: 1% | Biasanya tidak ramai\...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,08.00-15.30,*****,Senin,*****,*****,"-7.800293,110.3661642","11.00, 12.00, 13.00\n---\nTime Label: 11.00\nA...",Tuesday:\n 06.00: 0% | Idle\n 07.00: 0% ...,"Kursi khusus pengguna kursi roda, Pintu masuk ...",Tidak menerima reservasi
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,*****,NaN,NaN,NaN,NaN,NaN
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,Buka 24 jam,*****,Open All Days,*****,*****,"-7.802505999999999,110.37030759999999","19.00, 20.00, 18.00\n---\nTime Label: 19.00\nA...",Monday:\n 04.00: 29% | Biasanya tidak terla...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi


### children_enabled

In [31]:
def update_rating(data1, data2):
    # Loop melalui setiap baris di data1
    for idx, row in data1.iterrows():
        # Cari baris di data2 dengan nama_wisata yang sama
        matching_row = data2[data2['name'] == row['name']]
        
        if not matching_row.empty:
            print('Ada data', row['name'])
            # Jika ada kecocokan, perbarui jumlah_reviews di data1
            data1.at[idx, 'children_enabled'] = matching_row['children_enabled'].values[0]
        else:
            print('Tidak ada data', row['name'])
    
    # Kembalikan data1 yang sudah diperbarui
    return data1

updatedDf = update_rating(dfNew, dfByPlace)
updatedDf.head()

Ada data Titik Nol Yogyakarta
Ada data Kawasan Wisata Malioboro
Ada data Museum Benteng Vredeburg
Tidak ada data Kampung Wisata Taman Sari
Ada data Kampung Wisata Sayidan
Ada data Gereja Sayidan
Ada data Plengkung Gading
Ada data Embung Langensari
Ada data Taman Pintar Yogyakarta
Ada data Taman Pelangi Jogja
Tidak ada data KEBON BINATANG
Tidak ada data Keraton Ngayogyakarta Hadiningrat
Tidak ada data Kraton Jogja
Ada data Bringin Kembar Alun-Alun Selatan (Masangin) Yogyakarta
Tidak ada data Alun-Alun Lor Yogyakarta
Ada data DINO Adventure
Ada data Kampung Wisata Rejowinangun
Tidak ada data nan
Ada data Taman Kearifan UGM
Ada data Slasar Malioboro
Ada data Kampung Wisata Sosromenduran
Ada data Kampung Wisata Tahunan
Ada data Monumen Batik Yogyakarta
Ada data Jembatan Sayidan
Ada data Malioboro Yogyakarta Indonesia
Ada data Tebing Breksi
Ada data Bukit Bintang
Tidak ada data Puncak Pinus Becici
Ada data Bendhung Lepen
Ada data Candi Prambanan
Ada data Makam Panembahan Senopati
Ada data w

,place_id,name,reviews,main_category,categories,rating,address,link,query,description,...,is_temporarily_closed,closed_on,phone,review_keywords,coordinates,most_popular_times,popular_times,accessibility_enabled,planning_enabled,children_enabled
0,ChIJl5MfmllXei4RD-EMKO4ov-s,Titik Nol Yogyakarta,48855,Tujuan Wisata,"Tujuan Wisata, Pusat Rekreasi",4.8,"Jl. Pangurakan No.1, Ngupasan, Kec. Gondomanan...",https://www.google.com/maps/place/Titik+Nol+Yo...,tempat wisata di yogyakarta,*****,...,*****,Open All Days,*****,*****,"-7.8013965999999995,110.36476409999999","20.00, 21.00, 19.00\n---\nTime Label: 20.00\nA...",Monday:\n 04.00: 2% | Biasanya tidak ramai\...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak
1,ChIJ8TjDGYhXei4Rts_Mftk6qnI,Kawasan Wisata Malioboro,5190,Tujuan Wisata,Tujuan Wisata,4.7,"Jl. Margo Mulyo Jl. Jend. Ahmad Yani No.106, N...",https://www.google.com/maps/place/Kawasan+Wisa...,tempat wisata di yogyakarta,*****,...,*****,Open All Days,*****,*****,"-7.798248699999999,110.3652793","21.00, 20.00, 19.00\n---\nTime Label: 21.00\nA...",Monday:\n 04.00: 1% | Biasanya tidak ramai\...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak
2,ChIJz-6zwIhXei4R-J8qI-AcYbk,Museum Benteng Vredeburg,18416,Museum Sejarah,"Museum Sejarah, Benteng, Museum Ilmu Pengetahu...",4.6,"Jl. Margo Mulyo No.6, Ngupasan, Kec. Gondomana...",https://www.google.com/maps/place/Museum+Bente...,tempat wisata di yogyakarta,*****,...,*****,Senin,*****,*****,"-7.800293,110.3661642","11.00, 12.00, 13.00\n---\nTime Label: 11.00\nA...",Tuesday:\n 06.00: 0% | Idle\n 07.00: 0% ...,"Kursi khusus pengguna kursi roda, Pintu masuk ...",Tidak menerima reservasi,Cocok untuk anak-anak
3,ChIJK8_C0JNXei4RE74coPghaic,Kampung Wisata Taman Sari,41495,Bangunan Bersejarah,"Bangunan Bersejarah, Tujuan Wisata",4.6,"Patehan, Kecamatan Kraton, Kota Yogyakarta, Da...",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,*****,*****,*****,NaN,NaN,NaN,NaN,NaN,NaN
4,ChIJS4Sp_S5Xei4RD4jibwktlUI,Kampung Wisata Sayidan,20,Tujuan Wisata,Tujuan Wisata,4.9,"2 No.100, Jl. Sayidan GM No.2 no 63, RT.12/RW....",https://www.google.com/maps/place/Kampung+Wisa...,tempat wisata di yogyakarta,*****,...,*****,Open All Days,*****,*****,"-7.802505999999999,110.37030759999999","19.00, 20.00, 18.00\n---\nTime Label: 19.00\nA...",Monday:\n 04.00: 29% | Biasanya tidak terla...,"Pintu masuk khusus pengguna kursi roda, Tempat...",Tidak menerima reservasi,Cocok untuk anak-anak


In [35]:
updatedDf.to_csv('data/updated-data.csv')

# Data Cleaning

## Duplicated Data

In [36]:
updatedDf.duplicated().sum()

np.int64(0)

## Data Null

In [37]:
updatedDf.isna().sum()

place_id                  0
name                      1
reviews                   0
main_category             0
categories                0
rating                    0
address                   0
link                      0
query                     0
description               0
is_spending_on_ads        0
competitors               0
website                   0
can_claim                 0
owner_name                0
owner_profile_link        0
featured_image            0
workday_timing            0
is_temporarily_closed     0
closed_on                 0
phone                     0
review_keywords           0
coordinates              41
most_popular_times       41
popular_times            41
accessibility_enabled    41
planning_enabled         41
children_enabled         41
dtype: int64

## Feature Selection